In [1]:
import numpy as np
import dask
import fastprogress
from dask.distributed import Client

In [2]:
client = Client(threads_per_worker=1, n_workers=64)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 64
Total threads: 64,Total memory: 251.57 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34513,Workers: 64
Dashboard: http://127.0.0.1:8787/status,Total threads: 64
Started: Just now,Total memory: 251.57 GiB
Comm: tcp://127.0.0.1:43423,Total threads: 1
Dashboard: http://127.0.0.1:35341/status,Memory: 3.93 GiB
Nanny: tcp://127.0.0.1:38059,


In [3]:
np.random.seed = 42

In [5]:
n = 1260
num_tribes = 5
clan_size = 7
tribe_size = n // num_tribes
num_clans = tribe_size // clan_size

assert n == clan_size * num_clans * num_tribes
tribe_size, tribe_size/3

(252, 84.0)

In [6]:
def exp(num_runs,
        n = 1260,
        num_tribes = 5,
        f = 0.25,
        tribe_threshold = 84,
       ):

    total_failures = 0

    tribe_size = n // num_tribes # number of nodes in a tribe    
    num_byzantine_nodes = int(n * f) # total number of Byzantine nodes in the entire Network, including all tribes
            
    for i in range(num_runs):
        node_is_byzantine = np.concatenate((np.ones(num_byzantine_nodes, dtype=np.bool), # 1 means this node is Byzantine
                                    np.zeros(n - num_byzantine_nodes, dtype=np.bool))) # 0 means this node is honest
        np.random.shuffle(node_is_byzantine) # Byzantine nodes are randomly scattered
        structured_node_is_byzantine = node_is_byzantine.reshape([-1, tribe_size]) # (tribe, node) structure
        tribe_num_byzantine_nodes = structured_node_is_byzantine.sum(axis=1) # number of Byzantine nodes in each tribe

        tribe_is_byzantine = tribe_num_byzantine_nodes >= tribe_threshold # whether the i-th tribe is Byzantine
                
        total_failures += (tribe_is_byzantine.sum() > 0) # the simulation fails if there is at least one Byzantine tribe
    return total_failures

In [9]:
num_simulations = 100000000
num_tasks = 1000
num_runs = num_simulations // num_tasks

lazy_result_total = 0
for i in fastprogress.progress_bar(range(num_tasks)):
    lazy_result = dask.delayed(exp)(num_runs=num_runs, f=0.2)
    lazy_result_total += lazy_result

In [10]:
result = dask.compute(lazy_result_total) # watch http://server_ip:8787
result[0] / num_simulations

6e-08